In [2]:
from keras.layers import Embedding,LSTM,Dropout,Dense,Layer
from keras import Model,Input
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam
import keras.backend as K
import collections
import numpy as np
import time
from nltk.translate.bleu_score import corpus_bleu

class NmtModel(object):
  def __init__(self,source_dict,target_dict,use_attention):
    self.hidden_size = 200
    self.embedding_size = 100
    self.hidden_dropout_rate=0.2
    self.embedding_dropout_rate = 0.2
    self.batch_size = 100
    self.max_target_step = 30
    self.vocab_target_size = len(target_dict.vocab)
    self.vocab_source_size = len(source_dict.vocab)
    self.target_dict = target_dict
    self.source_dict = source_dict
    self.SOS = target_dict.word2ids['<start>']
    self.EOS = target_dict.word2ids['<end>']
    self.use_attention = use_attention

    print("source vocab: %d, target vocab:%d" % (self.vocab_source_size,self.vocab_target_size))


  def build(self):
    source_words = Input(shape=(None,),dtype='int32')
    target_words = Input(shape=(None,), dtype='int32')

    """
    Task 1 encoder
    
    Start
    """
    # implementing embedding layer and encoder

    # source embedding layer with input dim as source vocab size and output as embedding size
    # these are trainable layer with mask_zero = true to get rid of the padding
    embedding_source = Embedding(self.vocab_source_size, self.embedding_size, trainable=True, mask_zero = True)
    # target embedding layer with input dim as target vocab size and output as embedding size
    embedding_target = Embedding(self.vocab_target_size, self.embedding_size, trainable=True, mask_zero = True)
    # passing source words through the source embedding layer and adding dropout
    source_words_embeddings = embedding_source(source_words)
    source_words_embeddings = Dropout(self.embedding_dropout_rate)(source_words_embeddings)
    # passing target words through the target embedding layer and adding dropout
    target_words_embeddings = embedding_target(target_words)
    target_words_embeddings = Dropout(self.embedding_dropout_rate)(target_words_embeddings)
    # LSTM layer with units = self.hidden_size
    # return sequence is ture to get the output of all tokens
    # return state is true to get the hidden state and cell state from the encoder
    encoder_lstm = LSTM(self.hidden_size, recurrent_dropout=self.hidden_dropout_rate, return_sequences=True, return_state=True)
    # passing the source word embeddings to encoder_lstm
    encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(source_words_embeddings)
    
    """
    End Task 1
    """
    encoder_states = [encoder_state_h,encoder_state_c]

    decoder_lstm = LSTM(self.hidden_size,recurrent_dropout=self.hidden_dropout_rate,return_sequences=True,return_state=True)
    decoder_outputs_train,_,_ = decoder_lstm(target_words_embeddings,initial_state=encoder_states)


    if self.use_attention:
      decoder_attention = AttentionLayer()
      decoder_outputs_train = decoder_attention([encoder_outputs,decoder_outputs_train])

    decoder_dense = Dense(self.vocab_target_size,activation='softmax')
    decoder_outputs_train = decoder_dense(decoder_outputs_train)

    adam = Adam(lr=0.01,clipnorm=5.0)
    self.train_model = Model([source_words,target_words], decoder_outputs_train)
    self.train_model.compile(optimizer=adam,loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    self.train_model.summary()

    #Inference Models

    self.encoder_model = Model(source_words,[encoder_outputs,encoder_state_h,encoder_state_c])
    self.encoder_model.summary()

    decoder_state_input_h = Input(shape=(self.hidden_size,))
    decoder_state_input_c = Input(shape=(self.hidden_size,))
    encoder_outputs_input = Input(shape=(None,self.hidden_size,))

    """
    Task 2 decoder for inference
    
    Start
    """
    # putting decoder state input h and state input c in a list
    decoder_states = [decoder_state_input_h, decoder_state_input_c]
    # passing targer_words_embeddings through the decoder lstm
    # passing decoder states as the initial state for the decoder lstm
    decoder_outputs_test, decoder_state_output_h, decoder_state_output_c = decoder_lstm(target_words_embeddings,initial_state=decoder_states)
    # check if attention layer should be used
    # attention layer is used to provide the decoder with information from every encoder hidden state
    if self.use_attention:
      decoder_attention = AttentionLayer()
      decoder_outputs_test = decoder_attention([encoder_outputs_input,decoder_outputs_test])
    # passing output of the decoder lstm (attention = false) or decoder_attention (attention = true) to the decoder dense layer
    decoder_outputs_test = decoder_dense(decoder_outputs_test)
    
    """
    End Task 2 
    """

    self.decoder_model = Model([target_words,decoder_state_input_h,decoder_state_input_c,encoder_outputs_input],
                               [decoder_outputs_test,decoder_state_output_h,decoder_state_output_c])
    self.decoder_model.summary()



  def time_used(self, start_time):
    curr_time = time.time()
    used_time = curr_time-start_time
    m = used_time // 60
    s = used_time - 60 * m
    return "%d m %d s" % (m, s)

  def train(self,train_data,dev_data,test_data, epochs):
    start_time = time.time()
    for epoch in range(epochs):
      print("Starting training epoch {}/{}".format(epoch + 1, epochs))
      epoch_time = time.time()
      source_words_train, target_words_train, target_words_train_labels = train_data

      self.train_model.fit([source_words_train,target_words_train],target_words_train_labels,batch_size=self.batch_size)

      print("Time used for epoch {}: {}".format(epoch + 1, self.time_used(epoch_time)))
      dev_time = time.time()
      print("Evaluating on dev set after epoch {}/{}:".format(epoch + 1, epochs))
      self.eval(dev_data)
      print("Time used for evaluate on dev set: {}".format(self.time_used(dev_time)))

    print("Training finished!")
    print("Time used for training: {}".format(self.time_used(start_time)))

    print("Evaluating on test set:")
    test_time = time.time()
    self.eval(test_data)
    print("Time used for evaluate on test set: {}".format(self.time_used(test_time)))



  def get_target_sentences(self, sents,vocab,reference=False):
    str_sents = []
    num_sent, max_len = sents.shape
    for i in range(num_sent):
      str_sent = []
      for j in range(max_len):
        t = sents[i,j].item()
        if t == self.SOS:
          continue
        if t == self.EOS:
          break

        str_sent.append(vocab[t])
      if reference:
        str_sents.append([str_sent])
      else:
        str_sents.append(str_sent)
    return str_sents


  def eval(self, dataset):
    source_words, target_words_labels = dataset
    vocab = self.target_dict.vocab

    encoder_outputs, state_h,state_c = self.encoder_model.predict(source_words,batch_size=self.batch_size)
    predictions = []
    step_target_words = np.ones([source_words.shape[0],1]) * self.SOS
    for _ in range(self.max_target_step):
      step_decoder_outputs, state_h,state_c = self.decoder_model.predict([step_target_words,state_h,state_c,encoder_outputs],batch_size=self.batch_size)
      step_target_words = np.argmax(step_decoder_outputs,axis=2)
      predictions.append(step_target_words)

    candidates = self.get_target_sentences(np.concatenate(predictions,axis=1),vocab)
    references = self.get_target_sentences(target_words_labels,vocab,reference=True)
    score = corpus_bleu(references,candidates)
    print("Model BLEU score: %.2f" % (score*100.0))

    # code to print the first 5 sample outputs: predicted outputs are candidates and actual sentences are references
    print("===== PRINTING OUTPUT====")
    for i in range(5):
      print("{} Sample".format(i+1))
      print("Predicted Sentence: "+" ".join(candidates[i]))
      print("Actual Sentence: "+" ".join(references[i][0]))
    print("========================")


class AttentionLayer(Layer):
  def compute_mask(self, inputs, mask=None):
    if mask == None:
      return None
    return mask[1]

  def compute_output_shape(self, input_shape):
    return (input_shape[1][0],input_shape[1][1],input_shape[1][2]*2)


  def call(self, inputs, mask=None):
    encoder_outputs, decoder_outputs = inputs

    """
    Task 3 attention
    
    Start
    """
    # This attention mechanism computes the score between decoder_outpus and encoder_outputs by matrix multiplication
    # to multiply decoder_outputs and encoder_outputs, it is needed to transpose the last two dimensions of decoder_outputs
    # the last two dimensions are transposed as given below
    decoder_outputs_dim = K.permute_dimensions(decoder_outputs, (0,2,1))
    # luong_score is calculated by using a dot product  of encoder_outputs and decoder_outputs
    luong_score = K.batch_dot(encoder_outputs, decoder_outputs_dim, axes=None)
    # softmax is applied to get the attention score
    luong_score = K.softmax(luong_score, axis=1)
    # expand the dimensions so that encoder_outputs and luong score(attention score) has the same shape
    encoder_outputs = K.expand_dims(encoder_outputs, axis=2)
    luong_score = K.expand_dims(luong_score, axis=3)
    # encoded vector is created by doing element wise multiplication between encoder_outputs and luong_score
    encoder_vector = encoder_outputs * luong_score
    # sum the ecoder_vector along the axis = 1 (max_source_sent_len) to get the required encoder_vector 
    encoder_vector = K.sum(encoder_vector, axis=1)
    
    """
    End Task 3
    """
    # [batch,max_dec,2*emb]
    new_decoder_outputs = K.concatenate([decoder_outputs, encoder_vector])

    return new_decoder_outputs


class LanguageDict():
  def __init__(self, sents):
    word_counter = collections.Counter(tok.lower() for sent in sents for tok in sent)

    self.vocab = []
    self.vocab.append('<pad>') #zero paddings
    self.vocab.append('<unk>')
    self.vocab.extend([t for t,c in word_counter.items() if c > 10])

    self.word2ids = {w:id for id, w in enumerate(self.vocab)}
    self.UNK = self.word2ids['<unk>']
    self.PAD = self.word2ids['<pad>']



def load_dataset(source_path,target_path, max_num_examples=30000):
  source_lines = open(source_path).readlines()
  target_lines = open(target_path).readlines()
  assert len(source_lines) == len(target_lines)
  if max_num_examples > 0:
    max_num_examples = min(len(source_lines), max_num_examples)
    source_lines = source_lines[:max_num_examples]
    target_lines = target_lines[:max_num_examples]

  source_sents = [[tok.lower() for tok in sent.strip().split(' ')] for sent in source_lines]
  target_sents = [[tok.lower() for tok in sent.strip().split(' ')] for sent in target_lines]
  for sent in target_sents:
    sent.append('<end>')
    sent.insert(0,'<start>')

  source_lang_dict = LanguageDict(source_sents)
  target_lang_dict = LanguageDict(target_sents)

  unit = len(source_sents)//10

  source_words = [[source_lang_dict.word2ids.get(tok,source_lang_dict.UNK) for tok in sent] for sent in source_sents]
  source_words_train = pad_sequences(source_words[:8*unit],padding='post')
  source_words_dev = pad_sequences(source_words[8*unit:9*unit],padding='post')
  source_words_test = pad_sequences(source_words[9*unit:],padding='post')

  eos = target_lang_dict.word2ids['<end>']

  target_words = [[target_lang_dict.word2ids.get(tok,target_lang_dict.UNK) for tok in sent[:-1]] for sent in target_sents]
  target_words_train = pad_sequences(target_words[:8*unit],padding='post')

  target_words_train_labels = [sent[1:]+[eos] for sent in target_words[:8*unit]]
  target_words_train_labels = pad_sequences(target_words_train_labels,padding='post')
  target_words_train_labels = np.expand_dims(target_words_train_labels,axis=2)

  target_words_dev_labels = pad_sequences([sent[1:] + [eos] for sent in target_words[8 * unit:9 * unit]], padding='post')
  target_words_test_labels = pad_sequences([sent[1:] + [eos] for sent in target_words[9 * unit:]], padding='post')

  train_data = [source_words_train,target_words_train,target_words_train_labels]
  dev_data = [source_words_dev,target_words_dev_labels]
  test_data = [source_words_test,target_words_test_labels]

  return train_data,dev_data,test_data,source_lang_dict,target_lang_dict



if __name__ == '__main__':
  max_example = 30000
  use_attention = False
  train_data, dev_data, test_data, source_dict, target_dict = load_dataset("data.30.vi","data.30.en",max_num_examples=max_example)
  print("read %d/%d/%d train/dev/test batches" % (len(train_data[0]),len(dev_data[0]), len(test_data[0])))

  model = NmtModel(source_dict,target_dict,use_attention)
  model.build()
  model.train(train_data,dev_data,test_data,10)

read 24000/3000/3000 train/dev/test batches
source vocab: 2034, target vocab:2506
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 100)    203400      input_6[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 100)    250600      input_7[0][0]                    
__________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
24000/24000 [==============================] - 38s 2ms/step - loss: 2.1322 - accuracy: 0.2425
Time used for epoch 1: 0 m 39 s
Evaluating on dev set after epoch 1/10:
Model BLEU score: 1.65
===== PRINTING OUTPUT====
1 Sample
Predicted Sentence: and i &apos;m going to <unk> <unk> , and i &apos;m going to <unk> <unk> , and i &apos;m going to <unk> .
Actual Sentence: there are four <unk> <unk> that , each time this ring <unk> it , as it <unk> the <unk> of the display , it <unk> up a position signal .
2 Sample
Predicted Sentence: and i &apos;m going to <unk> .
Actual Sentence: but this is really just the beginning .
3 Sample
Predicted Sentence: the <unk> of the <unk> of <unk> .
Actual Sentence: it <unk> this by <unk> <unk> about two <unk> .
4 Sample
Predicted Sentence: and i &apos;m going to <unk> <unk> , and i &apos;m going to <unk> .
Actual Sentence: so as you can see here , this is a , <unk> <unk> <unk> board .
5 Sample
Predicted Sentence: the <unk> <unk> .
Actual Sentence: the